## Use albumentations library for image augmentation
To generate object detection dataset. Currently works for Pascal VOC format

Notebook author: yptheangel(Choo Wilson)

References:<br> 
https://www.curiousily.com/posts/image-data-augmentation-for-tensorflow-2-keras-and-pytorch-with-albumentations-in-python/
https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/example_bboxes.ipynb

In [22]:
!pip install albumentations

In [23]:
%matplotlib inline
import cv2
import os
import os.path as osp
import xml.etree.ElementTree as ET
import numpy as np
import albumentations as A
from albumentations import (Compose,BboxParams)
from matplotlib import pyplot as plt

In [24]:
def getObjects(fullname):
    CLS=['Stringing','Underextrusion']    #List class names in alphabetical order
    bb = ""
    objs=[]
    cls_ids=[]
    in_file = open(fullname)
    tree=ET.parse(in_file)
    root = tree.getroot()
    # iterate through the objects inside a xml file
    for i, obj in enumerate(root.iter('object')):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in CLS or int(difficult)==1:
            continue
        cls_id = CLS.index(cls)
        xmlbox = obj.find('bndbox')
        b = [int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text)]
        c_id = cls_id
        objs.append(b)
        cls_ids.append(c_id)
    return objs,cls_ids

In [25]:
# Functions to visualize bounding boxes and class labels on an image. 
# Based on https://github.com/facebookresearch/Detectron/blob/master/detectron/utils/vis.py

BOX_COLOR = (0, 200,0)
TEXT_COLOR = (0, 0, 0)

def visualize_bbox(img, bbox, class_id, class_idx_to_name, color=BOX_COLOR, thickness=2):
#     x_min, y_min, w, h = bbox
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color=color, thickness=thickness)
    class_name = class_idx_to_name[class_id]
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (int(x_min), int(y_min - int(1.3 * text_height))), (int(x_min + text_width), int(y_min)), BOX_COLOR, -1)
    cv2.putText(img, class_name, (int(x_min), int(y_min - int(0.3 * text_height))), cv2.FONT_HERSHEY_SIMPLEX, 0.35,TEXT_COLOR, lineType=cv2.LINE_AA)
    return img

def visualize(annotations, category_id_to_name):
    img = annotations['image'].copy()
    for idx, bbox in enumerate(annotations['bboxes']):
        img = visualize_bbox(img, bbox, annotations['category_id'][idx], category_id_to_name)
    plt.figure(figsize=(18, 32))
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [26]:
def get_aug(aug, min_area=0., min_visibility=0.):
       return Compose(aug, bbox_params=BboxParams(format='pascal_voc', min_area=min_area, min_visibility=min_visibility, label_fields=['category_id']))

In [36]:
doc_aug = [
    A.RandomCrop(p=0.2,height=500,width=500),
    A.RandomBrightnessContrast(p=0.6),
    A.Rotate(limit=10, p=0.4),
    A.VerticalFlip(p=0.3),
    A.HorizontalFlip(p=0.2),
    A.RGBShift(p=0.8),
    A.Blur(blur_limit=5, p=0.1),
    A.RandomScale(p=0.3),
    A.ChannelShuffle(p=0.5)
]

In [37]:
category_id_to_name = {0:'Stringing',1:'Underextrusion'}   #Change to class names

In [38]:
xml_body_1="""<annotation>
        <folder>FOLDER</folder>
        <filename>{FILENAME}</filename>
        <path>{PATH}</path>
        <source>
                <database>Unknown</database>
        </source>
        <size>
                <width>{WIDTH}</width>
                <height>{HEIGHT}</height>
                <depth>3</depth>
        </size>
"""
xml_object=""" <object>
                <name>{CLASS}</name>
                <pose>Unspecified</pose>
                <truncated>0</truncated>
                <difficult>0</difficult>
                <bndbox>
                        <xmin>{XMIN}</xmin>
                        <ymin>{YMIN}</ymin>
                        <xmax>{XMAX}</xmax>
                        <ymax>{YMAX}</ymax>
                </bndbox>
        </object>
"""
xml_body_2="""</annotation>        
"""

def save_to_voc(imgFile, xmlFile, aug, w, h):
    with open(xmlFile,"w") as f:
        f.write(xml_body_1.format(**{'FILENAME':os.path.basename(imgFile), 'PATH':imgFile,'WIDTH':w,'HEIGHT':h}))
        for i,bb in enumerate(aug['bboxes']):            
            f.write(xml_object.format(**{'CLASS':category_id_to_name[aug['category_id'][i]],'XMIN':int(aug['bboxes'][i][0]),'YMIN':int(aug['bboxes'][i][1]),'XMAX':int(aug['bboxes'][i][2]),'YMAX':int(aug['bboxes'][i][3])}))
        f.write(xml_body_2)

In [39]:
albumentationsError=0
datasetFolder=r'C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig'   #Set path to dataset
extensions = (".jpg",".JPG")  #Set the extensions of the dataset images
for root, dirs, files in os.walk(datasetFolder):
    for file in files:
        if file.endswith(extensions):
            imageFile = osp.join(root,file)
            xmlFile = osp.join(root,file[:-4]+".xml")
            print(imageFile)
            print(xmlFile)
            print("")
            objects=[]
            classIds=[]
            objects , classIds = getObjects(xmlFile)
            image = cv2.imread(imageFile)
            annotations = {'image':image,'bboxes':objects,'category_id':classIds}
# Generate how many augmented images for each image
            for i in range(4):
                try:
                    aug = get_aug(doc_aug)
                    augmented = aug(**annotations)
                    aug_h = augmented['image'].shape[0]
                    aug_w = augmented['image'].shape[1]
                    save_to_voc(f"{file[:-4]}_aug_{i}.jpg",f"{file[:-4]}_aug_{i}.xml",augmented,aug_w,aug_h)
                    cv2.imwrite(f"{file[:-4]}_aug_{i}.jpg",augmented['image'])
                except Exception as e:
                    print(e)
                    albumentationsError+=1

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (10).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (10).xml

Requested crop size (500, 500) is larger than the image size (472, 628)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (100).JPG
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (100).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (101).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (101).xml

Requested crop size (500, 500) is larger than the image size (768, 432)
Requested crop size (500, 500) is larger than the image size (768, 432)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (102).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (102).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (103).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (103).xml

R

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (38).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (38).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (39).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (39).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (4).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (4).xml

Requested crop size (500, 500) is larger than the image size (225, 400)
Requested crop size (500, 500) is larger than the image size (225, 400)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (40).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (40).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (41).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (41).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (42).jpg
C:\Use

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (86).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (86).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (87).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (87).xml

Requested crop size (500, 500) is larger than the image size (297, 450)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (88).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (88).xml

Requested crop size (500, 500) is larger than the image size (384, 512)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (89).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (89).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (9).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (9).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\STR (90).jpg
C:\Use

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (13).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (13).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (130).JPG
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (130).xml

Requested crop size (500, 500) is larger than the image size (231, 368)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (131).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (131).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (132).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (132).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (133).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (133).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (134).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (46).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (46).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (48).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (48).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (49).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (49).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (5).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (5).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (50).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (50).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (51).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (51).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (52)

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (94).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (94).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (95).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (95).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (96).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (96).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (97).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (97).xml

Requested crop size (500, 500) is larger than the image size (183, 276)
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (99).jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND (99).xml

C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND.jpg
C:\Users\nickj\Downloads\albumentations_scripts-master\Volledig\UND.xml



In [40]:
print(albumentationsError)

48
